## Business Analytics: Preprocessing Data for Tableau 
-----------------------------------------------------
Because our dataset is incredibly large, our Tableau dashboard, representing the user interface, will focus on individual user data. In order to also gain a greater understanding of the combined user data from a backend business analytics standpoint, we have decided to include a secondary dashboard representing the adminstrative interface. The following code snippet includes several preprocessing steps to extract the necessary information for our business interface without the computational intensity of including the entire raw dataset in Tableau. 

In [ ]:
# Importing necessary libraries 
import os
from pathlib import Path

import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean

First, let's combine the dataset, add 'user_id' as a column to retain individual data and use the assumption of negatives to fill in any missing values in the ground truth labels. 

In [2]:
# Accessing copy of stored files
directory = Path('Users/halladaykinsey/Desktop/ExtraSensory.per_uuid_features_labels-Unzipped copy')
dfs = []

# Iterating over each file in the directory
for file in directory.glob('*.csv'):
    df = pd.read_csv(file)
    
    # Adding user_id as column
    user_id = file.stem  
    df['user_id'] = user_id
    
    # Cleaning ground truth columns
    label_cols = df.columns[df.columns.str.startswith('label')]
    df[label_cols] = df[label_cols].fillna(0)
    
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)
print(combined_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377346 entries, 0 to 377345
Columns: 279 entries, timestamp to user_id
dtypes: float64(276), int64(2), object(1)
memory usage: 803.2+ MB
None


Next, let's find the user with the most and least number of missing values. In our business interface, this will be helpful for administrators to understand which users are missing specific sensors and which user prediction would be more accurate based on the number of sensor values being used. 

In [3]:
# Calculating missing values within input sensors for each user
input_cols = combined_df.columns[~combined_df.columns.str.startswith(('label', 'timestamp', 'user_id'))]
combined_df['missing_values_count'] = combined_df[input_cols].isnull().sum(axis=1)
missing_values_per_user = combined_df.groupby('user_id')['missing_values_count'].sum()

# Identifying users with the most and least missing values
user_with_most_missing_values = missing_values_per_user.idxmax()
user_with_least_missing_values = missing_values_per_user.idxmin()

print(f"User with the most missing values: {user_with_most_missing_values} - {missing_values_per_user.max()} missing values")
print(f"User with the least missing values: {user_with_least_missing_values} - {missing_values_per_user.min()} missing values")

User with the most missing values: 86A4F379-B305-473D-9D83-FC7D800180EF.features_labels - 765987 missing values
User with the least missing values: C48CE857-A0DD-4DDB-BEA5-3A25449B2153.features_labels - 50267 missing values


In [4]:
# Understanding timestamp to convert to datetime
print(combined_df['timestamp'].head())

0    1446141691
1    1446141752
2    1446141805
3    1446141873
4    1446141925
Name: timestamp, dtype: int64


In [5]:
# Converting Unix timestamps to datetime format
combined_df['timestamp'] = pd.to_datetime(combined_df['timestamp'], unit='s')

# Finding 'date of join' for each user
user_start_times = combined_df.groupby('user_id')['timestamp'].min().reset_index()
user_start_times.rename(columns={'timestamp': 'start_time'}, inplace=True)
print(user_start_times.head())

                                             user_id          start_time
0  00EABED2-271D-49D8-B599-1D4A09240601.features_... 2015-10-05 21:06:01
1  098A72A5-E3E5-4F54-A152-BBDA0DF7B694.features_... 2015-08-04 17:14:18
2  0A986513-7828-4D53-AA1F-E02D6DF9561B.features_... 2015-12-08 19:06:37
3  0BFC35E2-4817-4865-BFA7-764742302A2D.features_... 2015-10-20 18:42:14
4  0E6184E1-90C0-48EE-B25A-F1ECB7B9714E.features_... 2015-11-30 18:10:05


In [6]:
# Saving to CSV 
user_start_times.to_csv('/Users/halladaykinsey/Desktop/user_start_times.csv', index=False)
print("File saved successfully to desktop")

File saved successfully to desktop


Now that we have calculated the users with the most and least missing values, we can clean the rest of the input variables according to the threshold we used for our models in order to calculate similarity between users. 

In [7]:
# Calculating the percentage of missing data for each feature
total_nan_counts = combined_df.isnull().sum()
total_entries_per_feature = len(combined_df)
percentage_missing = (total_nan_counts / total_entries_per_feature) * 100

# Removing all input columns with missing values to maintain consistency with processed models
threshold = 0
columns_to_remove = percentage_missing[percentage_missing > threshold].index.tolist()
print(f"Removing {len(columns_to_remove)} columns out of {len(combined_df.columns)}")
combined_df.drop(columns=columns_to_remove, inplace=True)

Removing 191 columns out of 280


In [8]:
# Sorting through dataframe for included values 
data_for_similarity = combined_df.select_dtypes(include=['float64', 'int64', 'bool'])

# Calling unique user IDs
user_ids = combined_df['user_id'].unique()


min_distance = np.inf
min_users = (None, None)

# Iterating over each pair of users to calculate the Euclidean distance
for i in range(len(user_ids)):
    for j in range(i + 1, len(user_ids)):
        user1_data = data_for_similarity[combined_df['user_id'] == user_ids[i]]
        user2_data = data_for_similarity[combined_df['user_id'] == user_ids[j]]
        
        # Aligning data with columns
        if not user1_data.columns.equals(user2_data.columns):
            print(f"Skipping users {user_ids[i]} and {user_ids[j]} due to different columns")
            continue

        # Calculating Euclidean distance
        dist = euclidean(user1_data.mean(), user2_data.mean())
        
        if dist < min_distance:
            min_distance = dist
            min_users = (user_ids[i], user_ids[j])

print(f"The two most similar users are: {min_users} with a distance of {min_distance}")


The two most similar users are: ('11B5EC4D-4133-4289-B475-4E737182A406.features_labels', '2C32C23E-E30C-498A-8DD2-0EFB9150A02E.features_labels') with a distance of 0.49951046741197547


In [10]:
# Extracting the two most similar user data and saving to csv
user1_data = combined_df[combined_df['user_id'] == min_users[0]]
user2_data = combined_df[combined_df['user_id'] == min_users[1]]

user1_data.to_csv(f'/Users/halladaykinsey/Desktop/user_{min_users[0]}_data.csv', index=False)
user2_data.to_csv(f'/Users/halladaykinsey/Desktop/user_{min_users[1]}_data.csv', index=False)

print(f"Data for user {min_users[0]} saved to: /Users/halladaykinsey/Desktop/user_{min_users[0]}_data.csv")
print(f"Data for user {min_users[1]} saved to: /Users/halladaykinsey/Desktop/user_{min_users[1]}_data.csv")

Data for user 11B5EC4D-4133-4289-B475-4E737182A406.features_labels saved to: /Users/halladaykinsey/Desktop/user_11B5EC4D-4133-4289-B475-4E737182A406.features_labels_data.csv
Data for user 2C32C23E-E30C-498A-8DD2-0EFB9150A02E.features_labels saved to: /Users/halladaykinsey/Desktop/user_2C32C23E-E30C-498A-8DD2-0EFB9150A02E.features_labels_data.csv
